# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [3]:
spark = SparkSession.builder.appName('Linear_regression_consulting_project').getOrCreate()

In [6]:
df = spark.read.csv(r'F:\backup\practice\Apache_spark\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Linear_Regression\cruise_ship_info.csv',inferSchema=True,header=True)

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [9]:
indexer = StringIndexer(inputCol='Cruise_line',outputCol='Cruise_line_ind')
ind = indexer.fit(df).transform(df)
ind.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_ind|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [11]:
ind.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_ind']

In [40]:
assembler = VectorAssembler(inputCols=[
  'passengers',
 'length',
 'cabins',
'Cruise_line_ind'], outputCol = 'features')

In [41]:
output = assembler.transform(ind)

In [42]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_ind|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|[6.94,5.94,3.55,1...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|[6.94,5.94,3.55,1...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|[14.86,7.22,7.43,...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|[29.74,9.53,14.88...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8

In [43]:
labled_data = output.select('features','crew')
train_data, test_data = labled_data.randomSplit([0.7,0.3])

In [58]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               116|
|   mean| 8.080775862068965|
| stddev|3.4166546442420813|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [45]:
lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train_data)

In [46]:
Train_Summary = lr_model.summary

In [47]:
Train_Summary.pValues

[0.0031229319378422993,
 0.00016504918190252127,
 4.799050046244702e-12,
 0.02695202691064358,
 0.008048316580142156]

### Executing Backward Elimination method using P-value for elimination Number of passengers, Length of the ship, number of cabins and Cruise line are found significant

In [59]:
from pyspark.sql.functions import corr

In [63]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [64]:
df.select(corr('crew','Age')).show()

+-------------------+
|    corr(crew, Age)|
+-------------------+
|-0.5306565039638852|
+-------------------+



In [49]:
pred = lr_model.evaluate(test_data)

In [52]:
pred.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.4350604977567204|
|  0.4176201378215375|
|-0.10291740468770838|
|-0.25838295432335356|
|-0.06221810771667702|
|  0.4451632729956261|
| 0.09512802702318268|
| 0.09512802702318268|
| 0.42145996588191315|
| -0.7062430670048245|
| -0.7062430670048245|
|6.611259605699971E-4|
| -0.6536973378982696|
| -0.5187113114334805|
|  0.5109426351013155|
|-0.23432940226732946|
| 0.06940511902144042|
|-0.45355974624586715|
| -1.0361269709670564|
| -0.8774774964978844|
+--------------------+
only showing top 20 rows



In [53]:
pred.r2

0.9700479674893092

In [55]:
pred.meanSquaredError

0.3913618229980647

In [56]:
pred.rootMeanSquaredError

0.6255891806913422